<a href="https://colab.research.google.com/github/BrajanNieto/cough-analyzer-Covid/blob/main/01_P2_CovidCough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Proyecto de clasificación 'Identificacion de personas con covid'
 ----
  
  University : UTEC \\
  Course       : Machine Learning \\
  Professor    : Cristian López Del Alamo \\
  Topic        : SVM \\
  Deadline      : 06-07-2025
   

 ----

Write the names and surnames of the members and the percentage of participation of each one in the development of the practice:
 - Integrante 1: Lopez Medina Sebastian 100%
 - Integrante 2: Nieto Espinoza Brajan Esteban 100%
 - Integrante 3: Tapia Chasquibol Mateo 100%

 ----
 The objective of this project is to classify patients as either having COVID or not, using only the sound of the patient’s cough. For this, your group can use libraries to obtain the best feature vector to represent the sound of the
cough.

In [ ]:
# ============================================
# 1  Librerias y datos
# ============================================
# -------------------------------------------
# 1.1 Clonar repositorio
# -------------------------------------------
!git clone https://github.com/BrajanNieto/Mlearning.git
%cd Mlearning
# -------------------------------------------
# 1.2 Librerías
# -------------------------------------------
import os
import re
import math
import random
import itertools
#  Librerías extras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cvxopt
import librosa
import pywt
from scipy.signal import butter, filtfilt
# scikit-learn
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import resample
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

Cloning into 'Mlearning'...
remote: Enumerating objects: 1551, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 1551 (delta 1), reused 0 (delta 0), pack-reused 1546 (from 2)
Receiving objects: 100% (1551/1551), 417.26 MiB | 26.01 MiB/s, done.
Resolving deltas: 100% (265/265), done.
Updating files: 100% (1435/1435), done.
/content/Mlearning


## Audio Preprocessing y Extracción de Features

**Puntos clave a tener en cuenta:**

1. **`normalize_audio(audio)`**  
   - Escala el pico máximo de la señal a 0.99.  
   - Añade un pequeño epsilon (`1e-6`) para evitar división por cero.  

2. **`remove_silence(audio, sr, top_db=50)`**  
   - Usa `librosa.effects.split` para identificar segmentos con energía por encima de `top_db`.  
   - Reconstruye el audio concatenando solo los intervalos activos (sin silencio).

3. **`segment_audio(audio, sr, duration=1.5, hop=0.75)`**  
   - Crea fragmentos superpuestos de longitud fija (`duration` segundos) con un salto (`hop`) entre inicios.  
   - Calcula `frame_length = duration * sr` y `hop_length = hop * sr`.  
   - Genera una lista de segmentos para procesamiento batch.

4. **`extract_features(segment, sr)`**  
   - Calcula **128 MFCCs** con `n_fft=512` y `hop_length=256`, y luego obtiene su valor medio.  
   - Calcula la **tasa de cruce por cero** (zero-crossing rate) y toma su media.  
   - Concatena ambos para un vector final de características.

5. **Observación sobre dimensiones**  
   - Aunque el docstring menciona “vector de 125 dimensiones”, la concatenación de 128 MFCCs más 1 valor de ZCR da **129 dimensiones**. Comprueba que downstream acepte este tamaño.

In [ ]:
# ============================================
# 2  Funciones para  de Procesamiento de Audio
# ============================================

def normalize_audio(audio):
    """Normaliza el pico máximo a 0.99."""
    peak = np.max(np.abs(audio)) + 1e-6
    return audio / peak * 0.99

def remove_silence(audio, sr, top_db=50):
    """Recorta silencios basados en un umbral de decibelios."""
    intervals = librosa.effects.split(audio, top_db=top_db)
    return np.concatenate([audio[s:e] for s, e in intervals])

def segment_audio(audio, sr, duration=1.5, hop=0.75):
    """Divide en fragmentos superpuestos de duración fija."""
    fl = int(duration * sr)
    hl = int(hop * sr)
    return [audio[i:i+fl] for i in range(0, len(audio)-fl, hl)]

def extract_features(segment, sr):
    """Calcula MFCCs (128) y zero‐crossing rate (1) y devuelve vector de 125 dims."""
    mfccs = librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=128, n_fft=512, hop_length=256)
    zcr   = librosa.feature.zero_crossing_rate(segment)
    return np.hstack([mfccs.mean(axis=1), zcr.mean()])

## Creación de DataFrame de Features

- Recorre carpetas `Negative`, `Positive` y `Unknown`, cargando sólo `.wav`.  
- Normaliza audio y elimina silencios antes de segmentar.  
- Segmenta en ventanas superpuestas, extrae 128 MFCCs + ZCR por segmento.  
- Genera filas `[filename, label, mfcc…, zcr]`, filtra `Unknown`, no se tomara en cuenta como una categoria adicional y mapea tiquetas a 0/1.  


In [ ]:
# ============================================
# 3  Generación de registros y DataFrame
# ============================================
base_path = 'cleaned_data'
records = []
for label in ['Negative', 'Positive', 'Unknown']:
    folder = os.path.join(base_path, label)
    for fname in os.listdir(folder):
        if not fname.lower().endswith('.wav'):
            continue
        path = os.path.join(folder, fname)
        audio, sr = librosa.load(path, sr=16000)
        # 3.1. Preprocesamiento
        audio = normalize_audio(audio)
        audio = remove_silence(audio, sr)
        # 3.2. Segmentación
        segments = segment_audio(audio, sr)
        # 3.3. Extracción de features de cada segmento
        for seg in segments:
            feats = extract_features(seg, sr)
            records.append([fname, label] + feats.tolist())

# 3.4. Crear DataFrame con 'filename' y 'label'
cols = ['filename', 'label'] + [f'mfcc_{i}' for i in range(128)] + ['zcr']
df = pd.DataFrame(records, columns=cols)

#3.5
df = df[df['label'] != 'Unknown'].copy()
mapping = {'Negative': 0, 'Positive': 1}
df['label'] = df['label'].map(mapping)

print(df.head())

                     filename  label      mfcc_0     mfcc_1     mfcc_2  \
0  602_Negative_female_24.wav      0 -372.318604  57.342354 -45.061600   
1    866_Negative_male_29.wav      0 -357.252380  47.288013 -13.204280   
2    866_Negative_male_29.wav      0 -364.362854  42.072475 -17.226746   
3    866_Negative_male_29.wav      0 -385.721832  33.958363 -14.124773   
4    866_Negative_male_29.wav      0 -389.195892  31.053137 -16.161470   

      mfcc_3     mfcc_4     mfcc_5     mfcc_6     mfcc_7  ...  mfcc_119  \
0 -17.433853 -44.516247  -0.497882 -16.115273 -24.031391  ... -0.509453   
1  12.651052 -27.716703   3.609981 -35.844666 -12.021462  ... -0.691036   
2  12.977987 -28.532749   8.233962 -43.290474  -8.629445  ... -0.800812   
3  26.623161 -12.464399  15.497139 -44.528969  -5.281589  ... -0.531144   
4  21.019720 -12.703656  11.125703 -44.778397  -5.335186  ... -0.324802   

   mfcc_120  mfcc_121  mfcc_122  mfcc_123  mfcc_124  mfcc_125  mfcc_126  \
0 -0.365180 -0.550728 -0.1654

## Split de Datos por Archivo

- Usa `GroupShuffleSplit` para dividir el 80% train y 20% test garantizando que segmentos del mismo archivo no se mezclen y que no haya overfiting.  
- Genera `train_df` y `test_df` con índices reestablecidos.  
- Añade columna `sex`: si el nombre contiene “female” (no sensible a mayúsculas) → 1; si no → 0.  
- Elimina la columna `filename` al final.  


In [ ]:

# ============================================
# 4  Split a nivel de archivo
# ============================================

# 4.1 Creamos un splitter que separa por grupos (filename)
splitter = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(
    splitter.split(df, y=df['label'], groups=df['filename'])
)

train_df = df.iloc[train_idx].reset_index(drop=True)
test_df  = df.iloc[test_idx].reset_index(drop=True)
# 4.2
for subset in (train_df, test_df):
    # 1 = female, 0 = male (u otros)
    subset['sex'] = np.where(
        subset['filename'].str.contains('female', case=False),
        1, 0
    )
    subset.drop(columns=['filename'], inplace=True)


## Preparación de Conjuntos de Entrenamiento y Prueba

- Separa **features** (`X`) y **labels** (`y`) en `train_df` y `test_df`.  
- Elimina la columna `label` de `X_train` y `X_test`.  
- Muestra dimensiones de cada set y conteo de clases (`value_counts`).  


In [ ]:
# ============================================
# 5  Preparar X e y para entrenamiento y prueba
# ============================================
X_train = train_df.drop(columns='label')
y_train = train_df['label']

X_test  = test_df.drop(columns='label')
y_test  = test_df['label']

# 7. Vista previa (opcional)
print("Train set:", X_train.shape, y_train.value_counts().to_dict())
print("Test  set:", X_test.shape,  y_test.value_counts().to_dict())

Train set: (3969, 130) {0: 3673, 1: 296}
Test  set: (929, 130) {0: 850, 1: 79}


## Configuración de Modelos y Búsqueda de Hiperparámetros

- **Pipelines**  
  - Escalado con `StandardScaler` para regresión logística, SVM y KNN; árbol de decisión sin escalado.  
  - Todos los clasificadores usan `class_weight='balanced'` salvo KNN.

- **Modelos incluidos**  
  - **LogReg**: regresión logística (solver lbfgs, iteraciones 1000)  
  - **SVM**: máquina de vectores de soporte con núcleo RBF (probabilidad desactivada)  
  - **DT**: árbol de decisión  
  - **KNN**: k vecinos más cercanos

- **Grillas de hiperparámetros**  
  - **LogReg**: búsqueda de constantes de regularización `C` (valores de 0.01 a 100)  
  - **SVM**: `C` (0.1–10) y `gamma` (‘scale’ o 0.1)  
  - **DT**: profundidades de 2 a 100 (o ilimitada) y muestras mínimas por hoja 1–10  
  - **KNN**: número de vecinos entre 2 y 30


In [ ]:
# -------------------------------------------
# 6. Definición de pipelines y grids
# -------------------------------------------
base_pipes = {
    'LogReg': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(max_iter=1_000, solver='lbfgs',
                                   class_weight='balanced', random_state=42))
    ]),
    'SVM': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', SVC(kernel='rbf', class_weight='balanced', probability=False,
                    random_state=42))
    ]),
    'DT': Pipeline([
        ('clf', DecisionTreeClassifier(class_weight='balanced', random_state=42))
    ]),
    'KNN': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', KNeighborsClassifier())   # KNN no usa class_weight
    ])
}

param_grids = {
    'LogReg': {'clf__C': [0.01, 0.1, 1, 10, 100, 0.5, 0.6, 0.7]},
    'SVM'   : {'clf__C': [0.1, 1, 1.5, 10],
               'clf__gamma': ['scale', 0.1]},
    'DT'    : {'clf__max_depth': [None, 5, 10],
               'clf__min_samples_leaf': [2, 3, 5]},
    'KNN'   : {'clf__n_neighbors': [2, 3, 4, 10, 20, 30]}
}

## Evaluación con Grid Search y Validación Cruzada

- Itera sobre cada pipeline en `base_pipes`.  
- Usa `StratifiedKFold(n_splits=10, shuffle=True)` como CV interno.  
- Configura `GridSearchCV` con:
  - `scoring='f1_weighted'`  
  - `n_jobs=-1` para paralelizar  
  - `refit=True` para conservar el mejor modelo.  
- Ajusta (`fit`) en `X_train`, `y_train` y almacena el mejor estimador en `best_estimators[name]`.  
- Imprime el mejor `f1` obtenido y los parámetros óptimos (`best_params_`).  


In [ ]:
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import precision_recall_fscore_support
# -------------------------------------------
# 7. Grid Search + Bootstrap
# -------------------------------------------
B = 10  # número de remuestreos bootstrap
best_estimators = {}

for name, pipe in base_pipes.items():
    print(f"\nModelo: {name}")
    print("-" * 80)
    best_score = -np.inf
    best_params = None

    # Recorremos cada combinación de hiperparámetros
    for params in ParameterGrid(param_grids[name]):
        pipe.set_params(**params)
        boot_metrics = []

        # Generamos B muestras bootstrap
        for b in range(B):
            # Bootstrap sobre X_train, y_train
            X_bs, y_bs = resample(
                X_train, y_train,
                replace=True,
                random_state=42 + b
            )
            # Índices OOB (no seleccionados en bootstrap)
            oob_mask = ~X_train.index.isin(X_bs.index)
            X_oob, y_oob = X_train[oob_mask], y_train[oob_mask]
            if y_oob.shape[0] == 0:
                continue

            # Entrenamos y predecimos
            pipe.fit(X_bs, y_bs)
            y_pred = pipe.predict(X_oob)

            # Calculamos métricas weighted
            prec, rec, f1, _ = precision_recall_fscore_support(
                y_oob, y_pred,
                average='weighted',
                zero_division=0
            )
            boot_metrics.append((prec, rec, f1))

        # Si no hubo OOB válido, saltamos
        if not boot_metrics:
            continue

        # Media y desviación de cada métrica
        arr = np.array(boot_metrics)
        mean_prec, mean_rec, mean_f1 = arr.mean(axis=0)
        std_prec, std_rec, std_f1 = arr.std(axis=0)

        # Imprimimos resultados para esta combinación
        print(f"params = {params}")
        print(
            f"  f1   = {mean_f1:.3f} ±{std_f1:.3f}  |  "
            f"prec = {mean_prec:.3f} ±{std_prec:.3f}  |  "
            f"rec  = {mean_rec:.3f} ±{std_rec:.3f}"
        )

        # Actualizamos mejor modelo según f1 medio
        if mean_f1 > best_score:
            best_score = mean_f1
            best_params = params
            # Clonamos el pipe con esos parámetros
            from sklearn.base import clone
            best_estimators[name] = clone(pipe)

    print("-" * 80)
    print(
        f"✓ Mejor f1 medio = {best_score:.3f}  con params = {best_params}"
    )


Modelo: LogReg
--------------------------------------------------------------------------------
params = {'clf__C': 0.01}
  f1   = 0.839 ±0.009  |  prec = 0.913 ±0.006  |  rec  = 0.797 ±0.013
params = {'clf__C': 0.1}
  f1   = 0.839 ±0.008  |  prec = 0.911 ±0.007  |  rec  = 0.798 ±0.012
params = {'clf__C': 1}
  f1   = 0.839 ±0.007  |  prec = 0.909 ±0.008  |  rec  = 0.797 ±0.010
params = {'clf__C': 10}
  f1   = 0.839 ±0.007  |  prec = 0.909 ±0.008  |  rec  = 0.798 ±0.010
params = {'clf__C': 100}
  f1   = 0.839 ±0.007  |  prec = 0.909 ±0.008  |  rec  = 0.798 ±0.010
params = {'clf__C': 0.5}
  f1   = 0.839 ±0.008  |  prec = 0.910 ±0.008  |  rec  = 0.797 ±0.011
params = {'clf__C': 0.6}
  f1   = 0.839 ±0.007  |  prec = 0.909 ±0.008  |  rec  = 0.798 ±0.011
params = {'clf__C': 0.7}
  f1   = 0.839 ±0.007  |  prec = 0.909 ±0.008  |  rec  = 0.798 ±0.010
--------------------------------------------------------------------------------
✓ Mejor f1 medio = 0.839  con params = {'clf__C': 0.01}

Modelo:

## Evaluación en Conjunto de Prueba

- Para cada modelo óptimo en `best_estimators`:  
  - Predice etiquetas sobre `X_test`.  
  - Calcula **precision**, **recall** y **f1 ponderado**.  
- Almacena los resultados en una lista de diccionarios con claves `model`, `precision`, `recall`, `f1`.  
- Convierte la lista en un DataFrame, lo indexa por `model` y redondea a 3 decimales.  
- Imprime la tabla resultante en formato Markdown (`to_markdown()`).  


In [ ]:
# -------------------------------------------
# 8. Evaluación en test con los mejores estimadores
# -------------------------------------------
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

results_test = []

for name, est in best_estimators.items():
    # 0) Ajustamos el pipeline completo sobre X_train, y_train
    est.fit(X_train, y_train)

    # 1) Predecimos sobre X_test
    y_pred = est.predict(X_test)

    # 2) Calculamos precision, recall y f1 ponderado
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall    = recall_score(   y_test, y_pred, average='weighted', zero_division=0)
    f1        = f1_score(       y_test, y_pred, average='weighted', zero_division=0)

    results_test.append({
        'model':     name,
        'precision': precision,
        'recall':    recall,
        'f1':        f1,
    })

df_test = pd.DataFrame(results_test).set_index('model').round(3)
print(df_test.to_markdown())


| model   |   precision |   recall |    f1 |
|:--------|------------:|---------:|------:|
| LogReg  |       0.873 |    0.76  | 0.805 |
| SVM     |       0.916 |    0.926 | 0.904 |
| DT      |       0.866 |    0.87  | 0.868 |
| KNN     |       0.909 |    0.92  | 0.89  |


In [ ]:
svmb = best_estimators['SVM']
y_pred = svmb.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=svmb.classes_, columns=svmb.classes_)
print("Matriz de confusión (SVM):")
print(cm_df)
acc = accuracy_score(y_test, y_pred)
print(f"\nAccuracy total (SVM): {acc:.3f}")

Matriz de confusión (SVM):
     0   1
0  846   4
1   65  14

Accuracy total (SVM): 0.926


## Serializar y Guardar el Modelo SVM

- Se extrae el mejor estimador SVM: `svm_model = best_estimators['SVM']`.  
- Se usa `joblib.dump` para guardar el objeto en disco:  
  ```python
  joblib.dump(svm_model, 'svm_model.pkl')

In [ ]:
import joblib
svm_model = best_estimators['SVM']
joblib.dump(svm_model, 'svm_model.pkl')


['svm_model.pkl']

In [ ]:
#!apt-get update -qq
#!apt-get install -y libportaudio2 portaudio19-dev
#!pip install --upgrade --force-reinstall sounddevice

**USANDO EL ALGORITMO ENTRENADO:**  
Est a secci[on ofrece una interfaz de línea de comandos para predecir si un clip de tos es “Positive” o “Negative” para covid (etiqueta binaria) usando un modelo SVM previamente entrenado con características MFCC + ZCR.  

**Flujo principal:**  
1. **Parseo de argumentos**  
   - `--file`: ruta del audio (WAV, MP3, etc.)  
   - `--sex`: 0 (male/otro) o 1 (female)  
   - `--model`: archivo `.pkl` del modelo SVM  

2. **Carga del modelo**  
   - Valida existencia y carga con `joblib.load`.  

3. **Obtención del archivo de audio**  
   - Si no se pasa `--file`, permite subir en Colab o pide ruta local.  

7. **Salida de resultados**  
   - Muestra nombre de archivo, recuentos por segmento y porcentaje de confianza.  
   - Imprime mensaje final personalizado:  
     - Si etiqueta = 0 → “NO TIENES COVID!”  
     - Si etiqueta ≠ 0 → “SI TIENES COVID U.U”  


In [ ]:
"""
Inferencia de polaridad ('Positive'/'Negative') de un clip de voz
usando un modelo SVM entrenado con MFCC + ZCR.

✅ VERSIÓN SIMPLIFICADA - SOLO SUBIDA DE ARCHIVOS
-------------------------------------------------
1. Subir archivo de audio WAV
2. Extrae features con librosa
3. Ejecuta predicción con modelo SVM

Autor original: <tu nombre>
Última revisión: 29‑jun‑2025
"""

# ============================ Imports =======================================
import argparse
import os
import sys
from collections import Counter

import librosa
import numpy as np
import pandas as pd
from joblib import load

# ---------- Dependencias opcionales según entorno ----------
COLAB = False
try:
    import google.colab  # type: ignore
    from google.colab import files  # type: ignore
    COLAB = True
except ImportError:
    # Ejecución local
    pass

# ========================== 1. Pre‑procesamiento ============================

def normalize_audio(audio: np.ndarray) -> np.ndarray:
    """Normaliza a ±0.99 para evitar clipping."""
    peak = np.max(np.abs(audio)) + 1e-6
    return audio / peak * 0.99


def remove_silence(audio: np.ndarray, sr: int, top_db: int = 60) -> np.ndarray:
    """Elimina silencios usando librosa.effects.split."""
    intervals = librosa.effects.split(audio, top_db=top_db)
    return (
        np.concatenate([audio[s:e] for s, e in intervals])
        if intervals.size
        else audio
    )


def segment_audio(audio: np.ndarray, sr: int, duration: float = 1.5, hop: float = 0.75) -> list[np.ndarray]:
    """Ventanas deslizantes de `duration` s cada `hop` s."""
    fl, hl = int(duration * sr), int(hop * sr)
    return [audio[i : i + fl] for i in range(0, len(audio) - fl + 1, hl)]


def extract_features(segment: np.ndarray, sr: int) -> np.ndarray:
    """MFCC (128) + ZCR (1) → vector 129‑d."""
    mfccs = librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=128, n_fft=512, hop_length=256)
    zcr = librosa.feature.zero_crossing_rate(segment)
    return np.hstack([mfccs.mean(axis=1), zcr.mean()])

# ============================== 2. Modelo ====================================

def load_model(path: str):
    """Carga el modelo serializado con joblib/pickle."""
    if not os.path.isfile(path):
        sys.exit(f"⚠️  No existe el modelo: {path}")
    return load(path)

# ========================== 3. Audio helpers =================================

def upload_audio_colab() -> str:
    """Subir archivo de audio en Colab."""
    print("📁 Por favor, selecciona un archivo de audio WAV...")
    uploaded = files.upload()
    if not uploaded:
        sys.exit("⚠️  No se subió ningún archivo.")

    fname = next(iter(uploaded))
    if not fname.lower().endswith((".wav", ".mp3", ".m4a", ".flac")):
        print("⚠️ Formato no óptimo. Se recomienda WAV, pero se intentará procesar.")

    return fname

def get_audio_file_local() -> str:
    """Obtener archivo de audio en ejecución local."""
    while True:
        file_path = input("📁 Ruta del archivo de audio: ").strip()
        if os.path.isfile(file_path):
            return file_path
        print("⚠️ Archivo no encontrado. Intenta de nuevo.")

# =================== 4. Procesamiento de audio ===================

def process_audio_file(audio_path: str, sex: int) -> pd.DataFrame:
    """Procesa archivo de audio y extrae features."""
    print(f"🔊 Cargando audio: {audio_path}")

    # Cargar audio con librosa
    try:
        audio, sr = librosa.load(audio_path, sr=16_000)
        print(f"📊 Audio cargado: {len(audio)} samples a {sr} Hz ({len(audio)/sr:.2f} segundos)")
    except Exception as e:
        raise RuntimeError(f"❌ Error cargando audio: {e}")

    # Preprocessing
    audio = normalize_audio(audio)
    audio = remove_silence(audio, sr)

    if len(audio) < sr * 1.5:  # Menos de 1.5 segundos
        raise RuntimeError("⚠️ Audio muy corto después de quitar silencios")

    print(f"🔧 Audio procesado: {len(audio)/sr:.2f} segundos después de quitar silencios")

    # Segmentación
    segments = segment_audio(audio, sr)
    if not segments:
        raise RuntimeError("No se pudieron crear segmentos de audio")

    print(f"📊 Creados {len(segments)} segmentos para análisis")

    # Extracción de features
    features = []
    for i, segment in enumerate(segments):
        feat = extract_features(segment, sr)
        features.append(feat)
        if i < 3:  # Mostrar solo los primeros 3
            print(f"  Segmento {i+1}: MFCC promedio = {feat[:5].round(3)}")

    # Crear DataFrame
    df = pd.DataFrame(features, columns=[f"mfcc_{i}" for i in range(128)] + ["zcr"])
    df.insert(0, "sex", sex)
    df.insert(0, "filename", os.path.basename(audio_path))

    return df

def majority_vote(model, df_features: pd.DataFrame) -> tuple[str, Counter]:
    """Voto mayoritario sobre segmentos."""
    try:
        X = df_features[model.feature_names_in_]
    except AttributeError:
        X = df_features.drop(columns=["filename", "sex"], errors="ignore")

    preds = model.predict(X)
    tally = Counter(preds)
    return tally.most_common(1)[0][0], tally

# ====================== 5. CLI / argumentos =================================

def parse_args():
    ap = argparse.ArgumentParser(description="Inferencia sobre audio WAV")
    ap.add_argument("--file", help="Ruta del archivo de audio")
    ap.add_argument("--sex", choices=["0", "1"], help="0 = male/otro, 1 = female")
    ap.add_argument("--model", default="svm_model.pkl", help="Archivo del modelo SVM")
    args, _ = ap.parse_known_args()
    return args

# ========================== 6. Main =========================================

def main():
    args = parse_args()

    print("🤖 Cargando modelo SVM...")
    model = load_model(args.model)
    print("✅ Modelo cargado correctamente")

    # ---------- Obtener sexo ----------
    if args.sex is None:
        print("\n👤 Información del paciente:")
        while True:
            sex_input = input("Sexo (0 = masculino/otro, 1 = femenino): ").strip()
            if sex_input in ("0", "1"):
                args.sex = sex_input
                break
            print("⚠️ Por favor ingresa 0 ó 1")

    sex = int(args.sex)

    # ---------- Obtener archivo de audio ----------
    if args.file:
        # Archivo especificado por parámetro
        if not os.path.isfile(args.file):
            sys.exit(f"⚠️ Archivo no encontrado: {args.file}")
        audio_path = args.file
    else:
        # Obtener archivo según el entorno
        if COLAB:
            audio_path = upload_audio_colab()
        else:
            audio_path = get_audio_file_local()

    # ---------- Procesar audio ----------
    print(f"\n🔄 Procesando archivo: {audio_path}")
    df = process_audio_file(audio_path, sex)

    # ---------- Predicción ----------
    print("\n🧠 Ejecutando predicción...")
    final_label, counts = majority_vote(model, df)

    # ---------- Resultado ----------
    print("\n" + "=" * 60)
    print(f"📁 Archivo: {os.path.basename(audio_path)}")
    print(f"📊 Predicciones por segmento: {dict(counts)}")
    print(f"🎯 RESULTADO FINAL: {final_label}")
    if final_label == 0:
      print(f"🎯 RESULTADO FINAL: {final_label} – NO TIENES COVID!")
    else:
        print(f"🎯 RESULTADO FINAL: {final_label} – SI TIENES COVID U.U")
    print(f"📈 Confianza: {counts[final_label]}/{sum(counts.values())} segmentos")

    # Mostrar porcentaje de confianza
    confidence_pct = (counts[final_label] / sum(counts.values())) * 100
    print(f"💯 Porcentaje de segmentos: {confidence_pct:.1f}%")

    # Mostrar detalles por segmento si hay pocos
    if len(counts) <= 5:
        print("\n📋 Detalle por segmento:")
        for label, count in counts.most_common():
            pct = (count / sum(counts.values())) * 100
            print(f"  {label}: {count} segmentos ({pct:.1f}%)")

    print("=" * 60)


if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n❌ Cancelado por el usuario")
        sys.exit(0)
    except Exception as e:
        print(f"\n💥 Error: {e}")
        sys.exit(1)

🤖 Cargando modelo SVM...
✅ Modelo cargado correctamente

👤 Información del paciente:
Sexo (0 = masculino/otro, 1 = femenino): 0
📁 Por favor, selecciona un archivo de audio WAV...


Saving 179_Positive_male_25.wav to 179_Positive_male_25.wav

🔄 Procesando archivo: 179_Positive_male_25.wav
🔊 Cargando audio: 179_Positive_male_25.wav
📊 Audio cargado: 98304 samples a 16000 Hz (6.14 segundos)
🔧 Audio procesado: 5.86 segundos después de quitar silencios
📊 Creados 6 segmentos para análisis
  Segmento 1: MFCC promedio = [-459.693   89.929  -13.312   10.389   -9.594]
  Segmento 2: MFCC promedio = [-483.239   91.918   -5.656    5.673   -8.652]
  Segmento 3: MFCC promedio = [-527.178   99.269  -11.375   -2.381   -9.091]

🧠 Ejecutando predicción...

📁 Archivo: 179_Positive_male_25.wav
📊 Predicciones por segmento: {np.int64(1): 6}
🎯 RESULTADO FINAL: 1
🎯 RESULTADO FINAL: 1 – SI TIENES COVID U.U
📈 Confianza: 6/6 segmentos
💯 Porcentaje de segmentos: 100.0%

📋 Detalle por segmento:
  1: 6 segmentos (100.0%)
